# Inicio
## Import librarys

In [1]:
# ! conda install plotnine
# ! conda install pandas
# ! conda install scikit-learn
# ! conda install -c anaconda git


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from plotnine import (
ggplot,
geom_histogram,
aes,
geom_col,
theme,
element_text,
ggtitle,
scale_x_discrete,
facet_grid,
)

## Funciones utiles

In [4]:
import exploratory_lib as exp_lib

In [5]:
dic_med=exp_lib.class_diccionarios()
dic_med.set_list_num(list(range(1,13)))
dic_med.set_list_name(exp_lib.array_mes_name)
dic_med.set_dictionary()

In [6]:
dic_med.int_to_name(1)

'Enero'

## Extract CSV

In [7]:
# #elimina una columna en especifico
#     data_out.drop('Año natural', inplace=True, axis=1)
#     data_out.head(1)
df_origen = exp_lib.extract_csv("data","dataset_SCL",',')
df_origen.head(2)

Dimensiones: (68206, 18)
Columnas 
 Fecha-I, Vlo-I, Ori-I, Des-I, Emp-I, Fecha-O, Vlo-O, Ori-O, Des-O, Emp-O, DIA, MES, AÑO, DIANOM, TIPOVUELO, OPERA, SIGLAORI, SIGLADES


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami


## Transformacion de los datos

In [8]:
df_out = df_origen.copy().reset_index()
df_out=df_out.rename(columns= {'index':"id_vuelo"})

### Formato fecha

In [9]:
def convert_to_date(datetime_str):
    return pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M:%S')

In [10]:
convert_to_date("2017-01-02 23:39:00")

Timestamp('2017-01-02 23:39:00')

In [11]:
col_name_i = "Fecha-I"
col_name_0 = "Fecha-O"
df_out["date_i"] = df_out[col_name_i].apply(convert_to_date)
df_out["date_o"] = df_out[col_name_0].apply(convert_to_date)
df_out[[
    "date_i" , "Fecha-I",
    "date_o" , "Fecha-O",
]].head(3)

,date_i,Fecha-I,date_o,Fecha-O
0,2017-01-01 23:30:00,2017-01-01 23:30:00,2017-01-01 23:33:00,2017-01-01 23:33:00
1,2017-01-02 23:30:00,2017-01-02 23:30:00,2017-01-02 23:39:00,2017-01-02 23:39:00
2,2017-01-03 23:30:00,2017-01-03 23:30:00,2017-01-03 23:39:00,2017-01-03 23:39:00


### HIGH_SEASON

In [12]:
date_x = convert_to_date("2017-05-02 23:39:00")

Año_actual= date_x.year

# Between Dec-15 and Mar-3
verano_start = convert_to_date( f"{Año_actual}-12-15 00:00:00") 
verano_end = convert_to_date( f"{Año_actual}-3-4 00:00:00")
bool_verano = (
        (verano_end > date_x )
        |(date_x >verano_start)
        )
# Jul-15 and Jul-31, 
invierno_start = convert_to_date( f"{Año_actual}-7-15 00:00:00") 
invierno_end = convert_to_date( f"{Año_actual}-8-1 00:00:00")
bool_invierno = (invierno_end > date_x >invierno_start)
# Sep-11 and Sep-30
sep_start = convert_to_date( f"{Año_actual}-9-11 00:00:00") 
sep_end = convert_to_date( f"{Año_actual}-10-1 00:00:00")
bool_sep = (sep_end > date_x >sep_start)

# Condicion validado
high_season_value = 0
if bool_verano | bool_invierno | bool_sep:
    high_season_value =  1
else:
    high_season_value =  0
high_season_value

0

In [13]:
def col_high_season(date_x):
    Año_actual= date_x.year

    # Between Dec-15 and Mar-3
    verano_start = convert_to_date( f"{Año_actual}-12-15 00:00:00") 
    verano_end = convert_to_date( f"{Año_actual}-3-4 00:00:00")
    bool_verano = (
            (verano_end > date_x )
            |(date_x >verano_start)
            )
    # Jul-15 and Jul-31, 
    invierno_start = convert_to_date( f"{Año_actual}-7-15 00:00:00") 
    invierno_end = convert_to_date( f"{Año_actual}-8-1 00:00:00")
    bool_invierno = (invierno_end > date_x >invierno_start)
    # Sep-11 and Sep-30
    sep_start = convert_to_date( f"{Año_actual}-9-11 00:00:00") 
    sep_end = convert_to_date( f"{Año_actual}-10-1 00:00:00")
    bool_sep = (sep_end > date_x >sep_start)
    if bool_verano | bool_invierno | bool_sep:
        return 1
    else:
        return 0

In [14]:
df_out["high_season"] = df_out["date_i"].apply(col_high_season)
df_out[["high_season","date_i"]].head(3)

,high_season,date_i
0,1,2017-01-01 23:30:00
1,1,2017-01-02 23:30:00
2,1,2017-01-03 23:30:00


In [15]:
convert_to_date("2017-05-02 23:39:00").hour


23

### period_day

In [16]:
def period_day_func(hora_in):
    # morning (between 5:00 and 11:59)
    # afternoon (between 12:00 and 18:59) 
    # night (between 19:00 and 4:59)
    if 5<=hora_in<12:
        return "morning"
    elif 12<=hora_in<19:
        return "afternoon"
    elif (19<=hora_in) | (hora_in <5):
        return "night"
    else:
        return "error"

In [17]:
period_day_func(convert_to_date("2017-05-02 23:39:00").hour)

'night'

In [18]:
df_out["period_day"] = df_out["date_i"].dt.hour.apply(period_day_func)
df_out[["period_day","date_i"]].head(3)

,period_day,date_i
0,night,2017-01-01 23:30:00
1,night,2017-01-02 23:30:00
2,night,2017-01-03 23:30:00


### MIN_DIFF

In [19]:
df_out["min_diff"] = (df_out["date_o"] - df_out["date_i"]).dt.total_seconds() / 60
df_out[["min_diff",
"date_o",
"date_i",
]].head(3)

,min_diff,date_o,date_i
0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00


### delay_15

In [20]:
df_out["delay_15"] = df_out["min_diff"].apply( lambda x: 1 if x >15 else 0 )
df_out[["delay_15","min_diff","date_o","date_i",]].head(3)

,delay_15,min_diff,date_o,date_i
0,0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,0,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,0,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00


In [21]:
del df_out

## Proceso automatizado

In [22]:
df_total = df_origen.reset_index()
df_total=df_total.rename(columns= {'index':"id_vuelo"})
df_total = exp_lib.additional_columns(df_total)
df_total.head(2).T

,0,1
id_vuelo,0,1
Fecha-I,2017-01-01 23:30:00,2017-01-02 23:30:00
Vlo-I,226,226
Ori-I,SCEL,SCEL
Des-I,KMIA,KMIA
Emp-I,AAL,AAL
Fecha-O,2017-01-01 23:33:00,2017-01-02 23:39:00
Vlo-O,226,226
Ori-O,SCEL,SCEL
Des-O,KMIA,KMIA
